# 深度神经网络DNN模型

<a href='#dnnmodel'>一、DNN模型与前向传播算法</a>
- <a href='#1'>1. 从感知机到神经网络</a>
- <a href='#2'>2. DNN的基本结构</a>
- <a href='#3'>3. DNN前向传播算法的数学原理</a>
- <a href='#4'>4. DNN前向传播算法</a>
- <a href='#5'>5. DNN前向传播算法小结</a>

<a href='#bp'>二、DNN模型与反向传播算法BP</a>
- <a href='#bp1'>1. DNN反向传播算法要解决的问题</a>
- <a href='#bp2'>2. DNN反向传播算法的基本思路</a>
- <a href='#bp3'>3. DNN反向传播算法过程</a>
- <a href='#bp4'>4. DNN反向传播算法小结</a>

<a href='#cost_activation'>三、DNN模型与损失函数和激活函数的选择</a>
- <a href='#cost_activation1'>1. 均方差损失函数+Sigmoid激活函数的问题</a>
- <a href='#cost_activation2'>2. 使用交叉熵损失函数+Sigmoid激活函数改进DNN算法收敛速度</a>
- <a href='#cost_activation3'>3. 使用对数似然损失函数和softmax激活函数进行DNN分类输出</a>
- <a href='#cost_activation4'>4. 梯度爆炸、梯度消失与ReLU激活函数</a>
- <a href='#cost_activation5'>5. DNN其他激活函数</a>
- <a href='#cost_activation6'>6. DNN损失函数和激活函数小结</a>

<a href='#L'>四、DNN模型的正则化</a>
- <a href='#L1'>1. DNN的L1&L2正则化</a>
- <a href='#L2'>2. DNN通过集成学习的思路正则化</a>
- <a href='#L3'>3. DNN通过dropout 正则化</a>
- <a href='#L4'>4. DNN通过增强数据集正则化</a>
- <a href='#L5'>5. 其他DNN正则化方法</a>


# <a name='dnnmodel'>一、DNN模型与前向传播算法</a>

深度神经网络（Deep Neural Networks， 以下简称DNN）是深度学习的基础，而要理解DNN，首先我们要理解DNN模型，下面我们就对DNN的模型与前向传播算法做一个总结。

## <a name='1'>1. 从感知机到神经网络</a>

感知机的模型，它是一个有若干输入和一个输出的模型，如下图:
<img src='./images/percentron1.png' width='30%'/>

输出和输入之间学习到一个**线性关系**，得到中间输出结果：

$$z = \sum_{i=1}^{m} w_i x_i +b$$

接着是一个神经元激活函数:
$$sign(z) = \begin{cases}
-1 , z<0\\ 1, z \ge -1
\end{cases}$$

从而得到我们想要的输出结果1或者-1。

这个模型只能用于**二元分类**，且无法学习比较复杂的**非线性模型**，因此在工业界无法使用。

而神经网络则在感知机的模型上做了扩展，总结下主要有三点：   
    1. 加入了隐藏层，隐藏层可以有多层，增强模型的表达能力，如下图实例，当然增加了这么多隐藏层模型的复杂度也增加了好多。

<img src='./images/mlp1.png' width='50%'/>

    2. 输出层的神经元也可以不止一个输出，可以有多个输出，这样模型可以灵活的应用于分类回归，以及其他的机器学习领域，比如：降维和聚类等。多个神经元输出的输出层对应的一个实例。如下图，输出层现在有4个神经元了。
    
<img src='./images/mlp2.png' width='50%'/>

    3. 对激活函数做扩展，感知机的激活函数是sign(z)，虽然简单但是处理能力有限，因此神经网络中一般使用的其他的激活函数，比如我们在逻辑回归里面使用过的Sigmoid函数，即：
    
   $$f(z) = \frac{1}{1+e^{-z}}$$
   
    还有后来出现的tanx, softmax,和ReLU等。通过使用不同的激活函数，神经网络的表达能力进一步增强。对于各种常用的激活函数，我们在后面再专门讲。

## <a name='2'>2. DNN的基本结构</a>

上一节我们了解了神经网络基于感知机的扩展，主要在三方面：
- 添加隐藏层
- 单一输出变成多输出
- 激活函数：由简单的Sign(z)符号函数，添加多种变化的激活函数，如：Sigmoid, tanh, ReLU, softmax等


而DNN可以理解为有很多隐藏层的神经网络。这个很多其实也没有什么度量标准。   

多层神经网络和深度神经网络DNN其实也是指的一个东西，当然，DNN有时也叫做多层感知机（Multi-Layer perceptron,MLP）, 名字实在是多。

本文后面我们讲到的神经网络都默认为DNN。

从DNN按不同层的位置划分，DNN内部的神经网络层可以分为三类：
- 输入层
- 隐藏层
- 输出层

如下图示例，一般来说第一层是输入层，最后一层是输出层，而中间的层数都是隐藏层。

<img src='./images/dnn1.png' width='50%'/>

层与层之间是全连接的，也就是说，第i层的任意一个神经元一定与第i+1层的任意一个神经元相连。

虽然DNN看起来很复杂，但是从小的局部模型来说，还是和感知机一样，即一个线性关系$z=∑w_ix_i+b$加上一个激活函数σ(z)。

**由于DNN层数多，则我们的线性关系系数w和偏倚b的数量也就是很多了。具体的参数在DNN是如何定义的呢？**

首先，我们来看看线性关系系数$w$的定义。

以下图一个三层的DNN为例，**第二层的第4个神经元**到**第三层的第2个神经元**的线性系数定义为$w^3_{24}$。
- 上标3代表线性系数w所在的层数；
- 而下标对应的是输出的第三层索引2和输入的第二层索引4。

你也许会问，**为什么不是$w^3_{42}$, 而是$w^3_{24}$呢？**

这主要是为了便于模型用于矩阵表示运算，如果是$w^3_{42}$，而每次进行矩阵运算是$w^Tx+b$，需要进行转置。  
将输出的索引放在前面的话，则线性运算不用转置，即直接为$wx+b$。

总结下，第$l−1$层的第$k$个神经元到第$l$层的第$j$个神经元的线性系数定义为$w^l_{jk}$。  
注意，输入层是没有w参数的。

<img src='./images/dnn2.png' width='50%'/>

再来看看偏倚b的定义。

还是以这个三层的DNN为例，第二层的第三个神经元对应的偏倚定义为$b^2_3$。
其中：
- 上标2代表所在的层数
- 下标3代表偏倚所在的神经元的索引。

同样的道理，第三个的第一个神经元的偏倚应该表示为$b^3_1$。  
同样的，输入层是没有偏倚参数b的。

<img src='./images/dnn3.png' width='50%'/>

## <a name='3'>3. DNN前向传播算法数学原理</a>

在上一节，我们已经介绍了DNN各层线性关系系数w,偏倚b的定义。

假设我们选择的激活函数是σ(z)，隐藏层和输出层的输出值为a，则对于下图的三层DNN，利用和感知机一样的思路，我们可以利用上一层的输出计算下一层的输出，也就是所谓的DNN前向传播算法。

<img src='./images/dnnfp.png' width='50%'/>

对于第二层的输出$a^2_1$,$a^2_2$,$a^2_3$，我们有：
$$a^2_1 = \sigma(z_1^2) = \sigma(w^2_{11}x_1 + w^2_{12}x_2 + w^2_{13}x_3 + b_1^2  )$$
$$a^2_2 = \sigma(z_2^2) = \sigma(w^2_{21}x_1 + w^2_{22}x_2 + w^2_{23}x_3 + b_2^2  )$$
$$a^2_3 = \sigma(z_3^2) = \sigma(w^2_{31}x_1 + w^2_{32}x_2 + w^2_{33}x_3 + b_3^2  )$$

对于第三层的输出$a_1^3$，我们有：
$$a_1^3 = \sigma(z_1^3) = \sigma(w_{11}^3a^2_1 + w^3_{12}a^2_2+w^{3}_{13}a^2_3 + b^3_1)$$

将上面的例子一般化，假设第$l-1$层共有m个神经元，则对第$l$层的第$j$个神经元的输出$a^l_j$，我们有：
$$a^l_j = \sigma(z^l_j) = \sigma(\sum_{k=1}^m w_{jk}^l a_k^{l-1} + b^l_{j})$$

其中，如果$l=2$，则对于$a^1_k$即为输入层的$x_k$。

从上面可以看出，使用代数法一个个的表示输出比较复杂，而如果使用**矩阵法**则比较的简洁。



假设第$l-1$层共有m个神经元，而第$l$层有n个神经元，而第$l$层的线性系数$w$组成了一个$n*m$的矩阵$W^l$，第$l$层的偏倚$b$组成了一个$n*1$的向量$b^l$，第$l-1$层的输出$a$组成了一个$m*1$的向量$a^{l-1}$，第$l$层的未激活前线性输出$z$组成了一个$n*1$的向量$z_l$，第$l$层的输出$a$组成了一个$n*1$的向量$a^l$。 

则用矩阵表示，第$l$层的输出为：
$$a^l = \sigma(z^l) = \sigma(W^l a^{l-1} + b^l)$$

这个表示方法简洁漂亮，后面我们的讨论都会基于上面的这个矩阵法表示来。



<img src='./images/dnnfp2.png' width='50%'/>

## <a name='4'>4. DNN前向传播算法</a>

有了上一节的数学推导，DNN的前向传播算法也就不难了。

所谓的DNN的前向传播算法也就是利用我们的若干个权重系数矩阵W，偏倚向量b来和输入值向量x进行一系列**线性运算**和**激活运算**，从输入层开始，一层层的向后计算，一直到运算到输出层，得到输出结果为值。

输入: 总层数L，所有隐藏层和输出层对应的矩阵W，偏倚向量b，输入值向量x

输出：输出层的输出$a^L$

1. 初始化$a^1=x$
2. for l=2 to L, 计算：
$$a^l=σ(z^l)=σ(W^la^{l−1}+b^l)$$
最后的结果即为输出$a^L$。

## <a name='5'>5. DNN前向传播算法小结</a>

单独看DNN前向传播算法，似乎没有什么大用处，而且这一大堆的矩阵W，偏倚向量b对应的参数怎么获得呢？

**怎么得到最优的矩阵W，偏倚向量b呢？**

这个我们在讲DNN的**反向传播算法**时再讲。

而理解反向传播算法的前提就是理解DNN的模型与前向传播算法。这也是我们这一篇先讲的原因。

# <a name='bp'>二、DNN模型与反向传播算法BP</a>

在深度神经网络（DNN）模型与前向传播算法中，我们对DNN的模型和前向传播算法做了总结，这里我们更进一步，对DNN的反向传播算法（Back Propagation，BP）做一个总结。

## <a name='bp1'>1. DNN反向传播算法要解决的问题</a>


在了解DNN的反向传播算法前，我们**先要知道DNN反向传播算法要解决的问题，也就是说，什么时候我们需要这个反向传播算法？**　



回到我们监督学习的一般问题，假设我们有m个训练样本：$\{(x_1,y_1),(x_2,y_2),...,(x_m,y_m)\}$,其中$x$为输入向量，特征维度为$n_{in}$，而$y$为输出向量，特征维度为$n_{out}$。我们需要利用这m个样本训练出一个模型，当有一个新的测试样本$(x_{test},?)$来到时, 我们可以预测$y_{test}$向量的输出。

如果我们采用DNN的模型，即我们使输入层有$n_{in}$个神经元，而输出层有$n_{out}$个神经元。

再加上一些含有若干神经元的隐藏层。

此时我们**需要找到合适的所有隐藏层和输出层对应的线性系数矩阵W，偏倚向量b，让所有的训练样本输入计算出的输出尽可能的等于或很接近样本输出。**

**怎么找到合适的参数呢？**

如果大家对传统的机器学习的算法优化过程熟悉的话，这里就很容易联想到我们可以用一个**合适的损失函数**来**度量训练样本的输出损失**，接着对这个损失函数进行**优化求最小化的极值**，对应的一系列线性系数矩阵W，偏倚向量b即为我们的最终结果。

在DNN中，损失函数优化极值求解的过程最常见的一般是通过**梯度下降法**来一步步迭代完成的，当然也可以是其他的迭代方法。比如：**牛顿法**与**拟牛顿法**。

**对DNN的损失函数用梯度下降法进行迭代优化求极小值的过程**即为我们的**反向传播算法。**

## <a name='bp2'>2. DNN反向传播算法的基本思路</a>


在进行DNN反向传播算法前，我们需要选择一个损失函数，来度量训练样本计算出的输出和真实的训练样本输出之间的损失。

你也许会问：   
**训练样本计算出的输出是怎么得来的？**  
这个输出是随机选择一系列W,b,用我们上一节的前向传播算法计算出来的。即通过一系列的计算：$a^l=σ(z^l)=σ(W^la^{l−1}+b^l)$。计算到输出层第L层对应的$a^L$即为前向传播算法计算出来的输出。



回到损失函数，DNN可选择的损失函数有不少，为了专注算法，这里我们使用最常见的**均方差**来度量损失。

即对于每个样本，我们期望最小化下式：
$$J(W,b,x,y)=\frac{1}{2}||a^L-y||^2_2$$

其中，$a^L$和y为特征维度为$n_{out}$的向量，而$||S||_2$为S的L2范数。

损失函数有了，现在我们开始用梯度下降法迭代求解每一层的W,b。

首先是输出层第$L$层。注意到输出层的W,b满足下式：
$$a^L = \sigma(z^L) = \sigma(w^La^{L-1} +b^L)$$

这样对于输出层的参数，我们的损失函数变为：
$$J(W,b,x,y)=\frac{1}{2}||a^L-y||^2_2 = \frac{1}{2}||\sigma(w^La^{L-1} + b^L) -y||_2^2$$

这样，求解W和b的梯度就简单了：
$$\frac{\partial J(W,b,x,y)}{\partial W^L} = \frac{\partial J(W,b,x,y)}{\partial z^L} \frac{\partial z^L}{\partial W^L} = (a^L-y) \odot \sigma' (z^L)(a^{L-1})^T$$

$$\frac{\partial J(W,b,x,y)}{\partial b^L} = \frac{\partial J(W,b,x,y)}{\partial z^L} \frac{\partial z^L}{\partial b^L} = (a^L-y) \odot \sigma' (z^L)$$

注意，上式中有一个符号$\odot$，它代表Hadamard积，对于两个维度相同的向量$A(a_1,a_2,...,a_n)^T$和$B(b_1,b_2,...,b_n)^T$，则$A\odot B = (a_1 b_1, a_2b_2,...,a_nb_n)^T$。

计算推导过程见网页：https://www.cnblogs.com/pinard/p/6422831.html

我们注意到在求解输出层的W,b的时候，有公共的部分$\frac{\partial J(W,b,x,y)}{\partial z^L}$，因此我们可以把公共的部分，即对$z^L$先算出来，记为：
$$\delta^L = \frac{\partial J(W,b,x,y)}{\partial z^L} = (a^L-y) \odot \sigma'(z^L)$$

现在，我们终于把输出层的梯度算出来了，**那么如何计算上一层$L-1$的梯度？ 如何计算上上层$L-2$层的梯度呢？**

这里我们需要一步步的递推，注意到对于第$l$层的未激活输出$z^l$，它的梯度可以表示为：
$$\delta^l = \frac{\partial J(W,b,x,y)}{\partial z^l} = \frac{\partial J(W,b,x,y)}{\partial z^L}\frac{\partial z^L}{\partial z^{L-1}}\frac{\partial z^{L-1}}{\partial z^{L-2}}...\frac{\partial z^{l+1}}{\partial z^{l}}$$

**如果我们可以依次计算出第$l$层的$\delta^l$，则该层的$W^l$,$b^l$很容易计算？为什么呢？**

注意到根据前向传播算法，我们有：
$$z^l = W^l a^{l-1} + b^l$$

所以根据上式我们可以很方便的计算出第l层的$W^l$,$b^l$的梯度如下：

$$\frac{\partial J(W,b,x,y)}{\partial W^l} = \frac{\partial J(W,b,x,y)}{\partial z^l} \frac{\partial z^l}{\partial W^l} = \delta^l (a^{l-1})^T$$

$$\frac{\partial J(W,b,x,y)}{\partial b^l} = \frac{\partial J(W,b,x,y)}{\partial z^l} \frac{\partial z^l}{\partial b^l} = \delta^l$$

那么，现在**问题的关键就是求出$\delta^l$了。**

这里我们用到数学归纳法，第$L$层的$\delta^L$上面我们已经求出，假设第$l+1$层的$\delta^{l+1}$已经求出来了，那么我们如何求出第$l$层的$\delta^l$呢？

我们注意到：
$$\delta^l = \frac{\partial J(W,b,x,y)}{\partial z^{l}} $$
$$= \frac{\partial J(W,b,x,y)}{\partial z^{l+1}} \frac{\partial z^{l+1}}{\partial z^l} $$
$$=\delta^{l+1} \frac{\partial z^{l+1}}{\partial z^l} $$

可见，用归纳法递推$\delta^{l+1}$和$\delta^l$的关键在于求解$\frac{\partial z^{l+1}}{\partial z^l} $。

而$z^{l+1}$和$z^l$的关系其实很容易找出：
$$z^{l+1} = W^{l+1}a^l + b^{l+1} = W^{l+1}\sigma(z^l)+b^{l+1}$$

这样很容易求出：
<img src='./images/formula1.png' width='50%'/>

将上式带入上面$\delta^{l+1}$和$\delta^l$的关系式我们得到：

<img src='./images/formula2.png' width='50%'/>

现在我们得到了$δ^l$的递推关系式，只要求出了某一层的$δ^l$，求解$W^l$,$b^l$的对应梯度就很简单的。

## <a name='bp3'>3. DNN反向传播算法过程</a>


现在我们总结下DNN反向传播算法的过程。

由于梯度下降法有
- 批量（Batch），
- 小批量(mini-Batch)，
- 随机  
三个变种，为了简化描述，这里我们以最基本的批量梯度下降法为例来描述反向传播算法。

实际上在业界使用最多的是**mini-Batch的梯度下降法**。不过区别仅仅在于迭代时训练样本的选择而已。

输入: 总层数L，以及各隐藏层与输出层的神经元个数，激活函数，损失函数，迭代步长α，最大迭代次数MAX与停止迭代阈值ϵ，输入的m个训练样本$\{(x_1,y_1),(x_2,y_2),...,(x_m,y_m)\}$

输出：各隐藏层与输出层的线性关系系数矩阵W和偏倚向量b

1. 初始化各隐藏层与输出层的线性关系系数矩阵W和偏倚向量b的值为一个随机值。

2. for iter to 1 to MAX：  

2-1) for i =1 to m：   
    a) 将DNN输入$a^1$设置为$x_i$；  
    b) for l=2 to L，进行前向传播算法计算$a^{i,l}=σ(z……{i,l})=σ(W^la^{i,l−1}+b^l)$   
    c) 通过损失函数计算输出层的$δ^{i,L}$    
    d) for l= L to 2, 进行反向传播算法计算$δ^{i,l}=(W^{l+1})^Tδ^{i,l+1}⊙σ′(z^{i,l})$   
            
2-2) for l = 2 to L，更新第l层的$W^l,b^l$:
    $$W^l=W^l−α∑_{i=1}^mδ^{i,l}(a^{i,l−1})^T$$
    $$b^l=b^l−α∑_{i=1}^mδ^{i,l}$$
            如果所有W，b的变化值都小于停止迭代阈值ϵ，则跳出迭代循环到步骤3。
        


3. 输出各隐藏层与输出层的线性关系系数矩阵W和偏倚向量b。

## <a name='bp4'>4. DNN反向传播算法小结</a>

有了DNN反向传播算法，我们就可以很方便的用DNN的模型去解决第一节里面提到了各种监督学习的分类回归问题。

当然DNN的参数众多，矩阵运算量也很大，直接使用会有各种各样的问题。

有哪些问题以及如何尝试解决这些问题并优化DNN模型与算法，我们在下一篇讲。

# <a name='cost_activation'>三、DNN模型与损失函数和激活函数的选择</a>


在深度神经网络（DNN）反向传播算法(BP)中，我们对DNN的前向反向传播算法的使用做了总结。

里面使用的损失函数是均方差，而激活函数是Sigmoid。

实际上DNN可以使用的损失函数和激活函数不少。**这些损失函数和激活函数如何选择呢？**下面我们就对DNN损失函数和激活函数的选择做一个总结。

## <a name='cost_activation1'>1. 均方差损失函数+Sigmoid激活函数的问题</a>

在讲反向传播算法时，我们用均方差损失函数和Sigmoid激活函数做了实例，首先我们就来看看均方差+Sigmoid的组合有什么问题。

首先我们回顾下Sigmoid激活函数的表达式为：
$$\sigma(z)= \frac{1}{1+e^{-z}}$$

$\sigma(z)$的函数图像如下：
<img src='./images/sigmoid.png' width='50%'/>


从图上可以看出，对于Sigmoid，当z的取值越来越大后，函数曲线变得越来越平缓，意味着此时的导数σ′(z)也越来越小。同样的，当z的取值越来越小时，也有这个问题。仅仅在z取值为0附近时，导数σ′(z)的取值较大。


在上篇讲的均方差+Sigmoid的反向传播算法中，每一层向前递推都要乘以σ′(z),得到梯度变化值。

**Sigmoid的这个曲线意味着在大多数时候，我们的梯度变化值很小，导致我们的W,b更新到极值的速度较慢，也就是我们的算法收敛速度较慢。**

那么有什么什么办法可以改进呢？

## <a name='cost_activation2'>2. 使用交叉熵损失函数+Sigmoid激活函数改进DNN算法收敛速度</a>


上一节我们讲到Sigmoid的函数特性导致反向传播算法收敛速度慢的问题，那么如何改进呢？换掉Sigmoid？

这当然是一种选择。

**另一种常见的选择是用交叉熵损失函数来代替均方差损失函数。**

我们来看看每个样本的交叉熵损失函数的形式：
$$J(W,b,a,y) = -y \cdot \ln a - (1-y) \cdot \ln(1-a)$$

其中，$\cdot$是向量内积。 

这个损失函数的学名叫**交叉熵**。

使用了**交叉熵**损失函数，就**能解决Sigmoid函数导数变化大多数时候反向传播算法慢的问题吗？**


我们来看看当使用交叉熵时，我们输出层$δ^L$的梯度情况。

$$δ^L = \frac{\partial J(W,b,a^L, y)}{\partial z^L} $$
$$=-y \frac{1}{a^L}(1-a^L) + (1-y)\frac{1}{1-a^L}(a^L)(1-a^L)$$
$$=-y(1-a^L) + (1-y)a^L$$
$$=a^L-y$$

可见，此时我们的$δ^l$梯度表达式中里面已经没有了$\sigma'(z)$，作为一个特例，回顾我们上一节均方差损失函数时在$δ^L$梯度，
$$\frac{\partial J(W,b,x,y)}{\partial z^L} = (a^L-y) \odot \sigma'(z)$$

对比两者在第L层的$δ^L$的梯度表达式，就可以看出，使用交叉熵，得到的$δ^L$梯度表达式没有了$\sigma'(z)$，梯度为预测值和真实值的差距，这样求得的$W^l, b^l$的地图也不包含$\sigma'(z)$，因此，避免了反向传播收敛速度慢的问题。

通常情况下，**如果我们使用了sigmoid激活函数，交叉熵损失函数肯定比均方差损失函数好用。**

## <a name='cost_activation3'>3. 使用对数似然损失函数和softmax激活函数进行DNN分类输出</a>


在前面我们讲的所有DNN相关知识中，我们都假设输出是连续可导的值。

但是如果是**分类问题，那么输出是一个个的类别，那我们怎么用DNN来解决这个问题呢？**   

比如假设我们有一个三个类别的分类问题，这样我们的DNN输出层应该有三个神经元，假设第一个神经元对应类别一，第二个对应类别二，第三个对应类别三，这样我们期望的输出应该是(1,0,0)，（0,1,0）和(0,0,1)这三种。

即样本真实类别对应的神经元输出应该无限接近或者等于1，而非该样本真实输出对应的神经元的输出应该无限接近或者等于0。

或者说，**我们希望输出层的神经元对应的输出是若干个概率值，这若干个概率值即我们DNN模型对于输入值对于各类别的输出预测，同时为满足概率模型，这若干个概率值之和应该等于1。**



**DNN分类模型要求是输出层神经元输出的值在0到1之间，同时所有输出值之和为1。**

很明显，现有的普通DNN是无法满足这个要求的。   
但是我们只需要对现有的全连接DNN稍作改良，即可用于解决分类问题。

在现有的DNN模型中，我们可以**将输出层第i个神经元的激活函数**定义为如下形式：

$$a^L_i = \frac{e^{z_l^L}}{\sum_{j=1}^{nL}e^{z_j^L}}$$

其中，$nL$是输出层第L层的神经元个数，或者说我们分类问题的类别数。

很容易看出，所有的$a_i^L$都是在$(0,1)$之间的数字，而$\sum_{j=1}^{nL}$作为归一化因子保证了所有的$a^L_i$之和为1。

这个方法很简洁漂亮，仅仅只需要将输出层的激活函数从Sigmoid之类的函数转变为上式的激活函数即可。

上式这个激活函数就是我们的**softmax激活函数**。

它在分类问题中有广泛的应用。**将DNN用于分类问题，在输出层用softmax激活函数也是最常见的了。**

下面这个例子清晰的描述了softmax激活函数在前向传播算法时的使用。

假设我们的输出层为三个神经元，而未激活的输出为3,1和-3，我们求出各自的指数表达式为：20，2.7和0.05，我们的归一化因子即为22.75，这样我们就求出了三个类别的概率输出分布为0.88，0.12和0。

<img src='./images/softmax.png' width='80%'/>

从上面可以看出，**将softmax用于前向传播算法是也很简单的。那么在反向传播算法时还简单吗？反向传播的梯度好计算吗？答案是Yes！**

对于用于分类的softmax激活函数，对应的损失函数一般都是用对数似然函数，即：
$$J(W,b,a^L,y) = -\sum_k y_k \ln a^L_k$$

其中，$y_k$的取值为0或者1，如果某一训练样本的输出为第$i$类。则$y_i=1$，其余的$j≠i$都有$y_j=0$。

由于每个样本只属于一个类别，所以这个对数似然函数可以简化为：

$$J(W,b,a^L,y) = -\ln a^L_i$$

其中，i即为训练样本真实的类别序号。

可见，损失函数只和真实类别对应的输出有关，这样假设真实类别是第$i$类，则其他不属于第$i$类序号对应的神经元的梯度导数直接为0。  
对于真实类别第$i$类，他对应的第j个w链接wLij对应的梯度计算为：

$$\frac{\partial J(W,b,a^L, y)}{\partial w^L_{ij}} = \frac{\partial J(W,b,a^L,y)}{\partial a_i^L} \frac{\partial a^L_i}{\partial z^L_i} \frac{\partial z^L_i}{\partial w^L_{ij}}$$
$$=(a^L_i-1) a^{L-1}_j$$

同样的可以得到$b^L_i$的梯度表达式为：
$$\frac{\partial J(W,b,a^L,y)}{\partial b^L_i} = a^L_i -1$$

可见，梯度计算也很简洁，也没有第一节说的训练速度慢的问题。

举个例子，假如我们对于第2类的训练样本，通过前向算法计算的未激活输出为（1,5,3），则我们得到softmax激活后的概率输出为：(0.015,0.866,0.117)。

由于我们的类别是第二类，则反向传播的梯度应该为：(0.015,0.866-1,0.117)。

是不是很简单呢？是。

当softmax输出层的反向传播计算完以后，后面的普通DNN层的反向传播计算和之前讲的普通DNN没有区别。

## <a name='cost_activation4'>4. 梯度爆炸、梯度消失与ReLU激活函数</a>


学习DNN，大家一定听说过梯度爆炸和梯度消失两个词。**尤其是梯度消失，是限制DNN与深度学习的一个关键障碍，目前也没有完全攻克。**



**什么是梯度爆炸和梯度消失呢？**

从理论上说都可以写一篇论文出来。不过简单理解，就是在反向传播的算法过程中，由于我们使用了是矩阵求导的链式法则，有一大串连乘，如果连乘的数字在每层都是小于1的，则梯度越往前乘越小，导致梯度消失，而如果连乘的数字在每层都是大于1的，则梯度越往前乘越大，导致梯度爆炸。

In [7]:
import math
print(math.pow(0.99,10000)) # 这就是梯度消失的数学效果
print(math.pow(1.01, 10000)) # 这就是梯度爆炸的数学效果

2.2487748498162805e-44
1.6358287111890398e+43


比如我们在前一篇反向传播算法里面讲到了δ的计算，可以表示为：

$$δ^l=\frac{∂J(W,b,x,y)}{∂z^l}=\frac{∂J(W,b,x,y)}{∂z^L}\frac{∂z^L}{∂z^{L−1}}\frac{∂z^{L−1}}{∂z^{L−2}}...\frac{∂z^{l+1}}{∂z^l}$$

如果不巧我们的样本导致每一层$\frac{∂z^{l+1}}{∂z^l}$的都小于1，则随着反向传播算法的进行，我们的$δ^l$会随着层数越来越小，甚至接近越0，导致梯度几乎消失，进而导致前面的隐藏层的W,b参数随着迭代的进行，几乎没有大的改变，更谈不上收敛了。这个问题目前没有完美的解决办法。



而**对于梯度爆炸，则一般可以通过调整我们DNN模型中的初始化参数得以解决。**



对于**无法完美解决的梯度消失问题**，目前有很多研究，一个可能部分解决梯度消失问题的办法是**使用ReLU（Rectified Linear Unit）激活函数**，ReLU在卷积神经网络CNN中得到了广泛的应用，在CNN中梯度消失似乎不再是问题。那么它是什么样子呢？其实很简单，比我们前面提到的所有激活函数都简单，表达式为：
$$\sigma(z) =max(0,z)$$

也就是说大于等于0则不变，小于0则激活后为0。就这么一玩意就可以解决梯度消失？至少部分是的。具体的原因现在其实也没有从理论上得以证明。这里我也就不多说了。

## <a name='cost_activation5'>5. DNN其他激活函数</a>


除了上面提到了激活函数，DNN常用的激活函数还有：

1） tanh：这个是sigmoid的变种，表达式为：
$$tanh(z)=\frac{e^z−e^{−z}}{e^z+e^{-z}}$$

tanh激活函数和sigmoid激活函数的关系为：
$$tanh(z)=2sigmoid(2z)−1$$


tanh和sigmoid对比主要的特点是它的输出落在了[-1,1],这样输出可以进行标准化。

同时tanh的曲线在较大时变得平坦的幅度没有sigmoid那么大，这样求梯度变化值有一些优势。

当然，要说tanh一定比sigmoid好倒不一定，还是要具体问题具体分析。

2） softplus：这个其实就是sigmoid函数的原函数，表达式为：
$$softplus(z)=log(1+e^z)$$

它的导数就是sigmoid函数。softplus的函数图像和ReLU有些类似。它出现的比ReLU早，可以视为ReLU的鼻祖。

<img src='./images/softplus.png' width='50%'/>

3）PReLU：从名字就可以看出它是ReLU的变种，特点是如果未激活值小于0，不是简单粗暴的直接变为0，而是进行一定幅度的缩小。如下图。当然，由于ReLU的成功，有很多的跟风者，有其他各种变种ReLU，这里就不多提了。

<img src='./images/relu.png' width='50%'/>

## <a name='cost_activation6'>6. DNN损失函数和激活函数小结</a>

上面我们对DNN损失函数和激活函数做了详细的讨论，重要的点有：
- 如果使用sigmoid激活函数，则交叉熵损失函数一般肯定比均方差损失函数好。
- 如果是DNN用于分类，则一般在输出层使用softmax激活函数和对数似然损失函数。
- ReLU激活函数对梯度消失问题有一定程度的解决，尤其是在CNN模型中。

下一篇我们讨论下DNN模型的正则化问题。



# <a name='L'>四、DNN模型的正则化</a>




和普通的机器学习算法一样，DNN也会遇到过拟合的问题，需要考虑泛化，这里我们就对DNN的正则化方法做一个总结。

## <a name='L1'>1. DNN的L1&L2正则化</a>


想到正则化，我们首先想到的就是L1正则化和L2正则化。

L1正则化和L2正则化原理类似，这里重点讲述DNN的L2正则化。

而**DNN的L2正则化通常的做法是只针对与线性系数矩阵W,而不针对偏倚系数b。**

利用我们之前的机器学习的知识，我们很容易可以写出DNN的L2正则化的损失函数。


假如我们的每个样本的损失函数是均方差损失函数，则所有的m个样本的损失函数为：
$$J(W,b) =\frac{1}{2m} \sum_{i=1}^m ||a^L-y||^2_2$$

则加上了L2正则化后的损失函数是：
$$J(W,b) =\frac{1}{2m} \sum_{i=1}^m ||a^L-y||^2_2 + \frac{\lambda}{2m} \sum_{l=2}^L ||w||^2_2$$

其中，λ即我们的正则化超参数，实际使用时需要调参。  
而w为所有权重矩阵W的所有列向量。

如果使用上式的损失函数，进行反向传播算法时，流程和没有正则化的反向传播算法完全一样，区别仅仅在于进行梯度下降法时，W的更新公式。

回想我们在深度神经网络（DNN）反向传播算法(BP)中，W的梯度下降更新公式为：
$$W^l = W^l - \alpha \sum_{i=1}^m \delta^{i,l} (a^{x,l-1})^T$$


加入L2正则化以后，迭代更新公式变成：

$$W^l = W^l - \alpha \sum_{i=1}^m \delta^{i,l} (a^{x,l-1})^T-\alpha \lambda W^l$$

注意到上式中的梯度计算中$\frac{1}{m}$我忽略了，因为α是常数，而除以m也是常数，所以等同于用了新常数α来代替$\frac{α}{m}$。进而简化表达式，但是不影响损失算法。

类似的L2正则化方法可以用于交叉熵损失函数或者其他的DNN损失函数，这里就不累述了。

## <a name='L2'>2. DNN通过集成学习的思路正则化</a>


除了常见的L1&L2正则化，DNN还可以通过**集成学习的思路正则化**。

在集成学习原理小结中，我们讲到集成学习有Boosting和Bagging两种思路。

而**DNN可以用Bagging的思路来正则化**。 

常用的机器学习Bagging算法中，随机森林是最流行的。它 通过随机采样构建若干个相互独立的弱决策树学习器，最后采用加权平均法或者投票法决定集成的输出。在DNN中，我们一样使用Bagging的思路。不过和随机森林不同的是，我们这里不是若干个决策树，而是若干个DNN的网络。




首先我们要对原始的m个训练样本进行有放回随机采样，构建N组m个样本的数据集，然后分别用这N组数据集去训练我们的DNN。

即采用我们的前向传播算法和反向传播算法得到N个DNN模型的W,b参数组合，最后对N个DNN模型的输出用加权平均法或者投票法决定最终输出。



不过用集成学习Bagging的方法有一个问题，就是我们的DNN模型本来就比较复杂，参数很多。现在又变成了N个DNN模型，这样参数又增加了N倍，从而导致训练这样的网络要花更加多的时间和空间。因此一般N的个数不能太多，比如5-10个就可以了。

## <a name='L3'>3. DNN通过dropout 正则化</a>


这里我们再讲一种和Bagging类似但是又不同的正则化方法：Dropout。

所谓的Dropout指的是在用前向传播算法和反向传播算法训练DNN模型时，一批数据迭代时，随机的从全连接DNN网络中去掉一部分隐藏层的神经元。

比如我们本来的DNN模型对应的结构是这样的：

<img src='./images/beforedropout.png' width='50%'/>

在对训练集中的一批数据进行训练时，我们随机去掉一部分隐藏层的神经元，并用去掉隐藏层的神经元的网络来拟合我们的一批训练数据。

如下图，去掉了一半的隐藏层神经元：

<img src='./images/dropout.png' width='50%'/>

然后用这个去掉隐藏层的神经元的网络来进行一轮迭代，更新所有的W,b。这就是所谓的dropout。

当然，dropout并不意味着这些神经元永远的消失了。

在下一批数据迭代前，我们会把DNN模型恢复成最初的全连接模型，然后再用随机的方法去掉部分隐藏层的神经元，接着去迭代更新W,b。

当然，这次用随机的方法去掉部分隐藏层后的残缺DNN网络和上次的残缺DNN网络并不相同。

总结下dropout的方法：   
每轮梯度下降迭代时，它需要将训练数据分成若干批，然后分批进行迭代，每批数据迭代时，需要将原始的DNN模型随机去掉部分隐藏层的神经元，用残缺的DNN模型来迭代更新W,b。每批数据迭代更新完毕后，要将残缺的DNN模型恢复成原始的DNN模型。



从上面的描述可以看出dropout和Bagging的正则化思路还是很不相同的。

**dropout模型中的W,b是一套，共享的。所有的残缺DNN迭代时，更新的是同一组W,b；而Bagging正则化时每个DNN模型有自己独有的一套W,b参数，相互之间是独立的。**

当然他们每次使用基于原始数据集得到的分批的数据集来训练模型，这点是类似的。

使用基于dropout的正则化比基于bagging的正则化简单，这显而易见，当然天下没有免费的午餐，由于dropout会将原始数据分批迭代，因此原始数据集最好较大，否则模型可能会欠拟合。

## <a name='L4'>4. DNN通过增强数据集正则化</a>


增强模型泛化能力最好的办法是有**更多更多的训练数据**，但是在实际应用中，更多的训练数据往往很难得到。有时候我们不得不去自己想办法能无中生有，来增加训练数据集，进而得到让模型泛化能力更强的目的。

对于我们传统的机器学习分类回归方法，增强数据集还是很难的。你无中生有出一组特征输入，却很难知道对应的特征输出是什么。

但是对于DNN擅长的领域，比如图像识别，语音识别等则是有办法的。

以图像识别领域为例，对于原始的数据集中的图像，我们可以将原始图像稍微的平移或者旋转一点点，则得到了一个新的图像。虽然这是一个新的图像，即样本的特征是新的，但是我们知道对应的特征输出和之前未平移旋转的图像是一样的。

举个例子，下面这个图像，我们的特征输出是5。
<img src='./images/5.png' width='5%'/>

我们将原始的图像旋转15度，得到了一副新的图像如下：
<img src='./images/5_15.png' width='5%'/>

我们现在得到了一个新的训练样本，输入特征和之前的训练样本不同，但是特征输出是一样的，我们可以确定这是5.

用类似的思路，我们可以对原始的数据集进行增强，进而得到增强DNN模型的泛化能力的目的。

## <a name='L5'>5. 其他DNN正则化方法</a>

DNN的正则化的方法是很多的，还是持续的研究中。在Deep Learning这本书中，正则化是洋洋洒洒的一大章。

里面提到的其他正则化方法有：
- Noise Robustness，
- Adversarial Training，
- Early Stopping等。

如果大家对这些正则化方法感兴趣，可以去阅读Deep Learning这本书中的第七章。